loading toi headlines


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
dir_path = "/content/gdrive/My Drive/tasks/"
for dirname, _, filenames in os.walk(dir_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/gdrive/My Drive/tasks/india-news-headlines.csv
/content/gdrive/My Drive/tasks/sentiment_news.csv


In [ ]:
import pandas as pd
df=pd.read_csv(dir_path+"india-news-headlines.csv")
print(df.head(5))

   publish_date  ...                                      headline_text
0      20010101  ...  win over cena satisfying but defeating underta...
1      20010102  ...  Status quo will not be disturbed at Ayodhya; s...
2      20010102  ...                Fissures in Hurriyat over Pak visit
3      20010102  ...              America's unwanted heading for India?
4      20010102  ...                 For bigwigs; it is destination Goa

[5 rows x 3 columns]


Fixing Date

In [ ]:
df['publish_date']=df['publish_date'].apply(str)
df['publish_date']=df['publish_date'].apply(lambda x:x[:4]+"/"+x[4:6]+"/"+x[6:])
df['publish_date']=pd.to_datetime(df['publish_date'])
print(df)

        publish_date  ...                                      headline_text
0         2001-01-01  ...  win over cena satisfying but defeating underta...
1         2001-01-02  ...  Status quo will not be disturbed at Ayodhya; s...
2         2001-01-02  ...                Fissures in Hurriyat over Pak visit
3         2001-01-02  ...              America's unwanted heading for India?
4         2001-01-02  ...                 For bigwigs; it is destination Goa
...              ...  ...                                                ...
3297167   2020-06-30  ...      why tiktok removed 1 65 crore videos in india
3297168   2020-06-30  ...  apurva asrani calls alia bhatts mother soni ra...
3297169   2020-06-30  ...  kangana ranaut gets a doll version of herself ...
3297170   2020-06-30  ...  meezaan jaffrey reminisces his childhood days ...
3297171   2020-06-30  ...  prabhas20 titled as radhe shyam prabhas and po...

[3297172 rows x 3 columns]


In [ ]:
df=df[['publish_date','headline_text']]
dict_news={}

Grouping headlines date wise


In [ ]:
df.info()
temp = df.loc[0, 'publish_date']
temp2 = str(df.loc[0, 'headline_text'])
for x in range(1, len(df)):
    if df.loc[x, 'publish_date']==temp:
        temp2 += '. ' + str(df.loc[x, 'headline_text'])
    else:
        dict_news[df.loc[x-1, 'publish_date']] = temp2
        temp2 = ""
        temp = df.loc[x, 'publish_date']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3297172 entries, 0 to 3297171
Data columns (total 2 columns):
 #   Column         Dtype         
---  ------         -----         
 0   publish_date   datetime64[ns]
 1   headline_text  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 50.3+ MB


Making res data frame having headlines for each day

In [ ]:
import numpy as np
data={'headline':dict_news.values(),'publish_date':dict_news.keys()}
res=pd.DataFrame(data,index=np.arange(0,len(dict_news)))

In [ ]:
import nltk
nltk.download('all')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

Using Sentiment analyser of nltk toolkit to get headline sentiment

In [ ]:


sia = SIA()
results = []

for i in range(len(res)):
    #print(res.loc[i,'headline'])
    pol_score = sia.polarity_scores(res.loc[i,'headline'])
    pol_score['headline'] = res.loc[i,'headline']
    results.append(pol_score)

#print(results[:3], width=100)


TypeError: ignored

In [ ]:
out=pd.DataFrame()
for i in range(len(res)):
  out.loc[i,'headline']=res.loc[i,'headline']
  out.loc[i,'date']=res.loc[i,'publish_date']
  out.loc[i,'neg']=results[i]['neg']
  out.loc[i,'neu']=results[i]['neu']
  out.loc[i,'pos']=results[i]['pos']
  out.loc[i,'compound']=results[i]['compound']


In [ ]:
print(out)
out.to_csv("sentiment_news.csv")

                                               headline  ... compound
0     win over cena satisfying but defeating underta...  ...  -0.0000
1     . Fissures in Hurriyat over Pak visit. America...  ...  -0.9822
2     . Think again; Laxman tells Sangh Parivar. Hur...  ...   0.4329
3     . Vajpayee gives big push to tech education; R...  ...   0.8738
4     . Hawkings' day out. Light combat craft takes ...  ...   0.9666
...                                                 ...  ...      ...
7074  . Telangana: Senior Congress leader tests+ve f...  ...  -0.9999
7075  . up hammers out 60k proposals of rs 870cr ahe...  ...  -0.9999
7076  . Major Ravi to helm movie on Indo-Chinese con...  ...  -0.9999
7077  . Sports without spectators: 'Who will cheer f...  ...  -0.9999
7078  . Anna Eden gets creative with husband and MP ...  ...  -0.9999

[7079 rows x 6 columns]


Loading sentiment dataset produced previously

In [ ]:
import pandas as pd
df=pd.read_csv(dir_path+"sentiment_news.csv")
df=df.drop(columns=['Unnamed: 0'])
df['date']=pd.to_datetime(df['date'])
df


,headline,date,neg,neu,pos,compound
0,win over cena satisfying but defeating underta...,2001-01-01,0.230,0.473,0.297,-0.0000
1,. Fissures in Hurriyat over Pak visit. America...,2001-01-02,0.124,0.807,0.069,-0.9822
2,. Think again; Laxman tells Sangh Parivar. Hur...,2001-01-03,0.113,0.771,0.116,0.4329
3,. Vajpayee gives big push to tech education; R...,2001-01-04,0.092,0.812,0.096,0.8738
4,. Hawkings' day out. Light combat craft takes ...,2001-01-05,0.124,0.722,0.155,0.9666
...,...,...,...,...,...,...
7074,. Telangana: Senior Congress leader tests+ve f...,2020-06-25,0.137,0.801,0.062,-0.9999
7075,. up hammers out 60k proposals of rs 870cr ahe...,2020-06-26,0.160,0.751,0.089,-0.9999
7076,. Major Ravi to helm movie on Indo-Chinese con...,2020-06-27,0.158,0.750,0.092,-0.9999
7077,. Sports without spectators: 'Who will cheer f...,2020-06-28,0.148,0.779,0.074,-0.9999


In [ ]:
def categorise_compound(val):
  if val>0.3:
    return 1
  elif val<-0.3:
    return -1
  else:
    return 0    

Categorise Compund column 1=positive -1=negative 0=neutral

In [ ]:
df['compound']=df['compound'].apply(categorise_compound)
df.rename(columns = {'date':'Date'}, inplace = True)
df

,headline,Date,neg,neu,pos,compound
0,win over cena satisfying but defeating underta...,2001-01-01,0.230,0.473,0.297,0
1,. Fissures in Hurriyat over Pak visit. America...,2001-01-02,0.124,0.807,0.069,-1
2,. Think again; Laxman tells Sangh Parivar. Hur...,2001-01-03,0.113,0.771,0.116,1
3,. Vajpayee gives big push to tech education; R...,2001-01-04,0.092,0.812,0.096,1
4,. Hawkings' day out. Light combat craft takes ...,2001-01-05,0.124,0.722,0.155,1
...,...,...,...,...,...,...
7074,. Telangana: Senior Congress leader tests+ve f...,2020-06-25,0.137,0.801,0.062,-1
7075,. up hammers out 60k proposals of rs 870cr ahe...,2020-06-26,0.160,0.751,0.089,-1
7076,. Major Ravi to helm movie on Indo-Chinese con...,2020-06-27,0.158,0.750,0.092,-1
7077,. Sports without spectators: 'Who will cheer f...,2020-06-28,0.148,0.779,0.074,-1


In [ ]:
stocks=pd.read_csv(dir_path+"CSV.csv")
stocks.head(5)
len(stocks)

4902

Loading Stocks csv from yahoo.com

In [ ]:
print(stocks.describe())
print(stocks.info())

              Open         High  ...    Adj Close        Volume
count  4902.000000  4902.000000  ...  4902.000000  4.902000e+03
mean     11.625875    11.789761  ...    10.976573  7.103142e+04
std       8.472190     8.556532  ...     8.211427  8.540647e+04
min       1.312500     1.437500  ...     1.280911  0.000000e+00
25%       4.760000     4.830000  ...     4.324210  2.340000e+04
50%       6.600000     6.740000  ...     6.018464  4.870000e+04
75%      19.785001    20.049999  ...    19.034070  8.927500e+04
max      29.000000    29.110001  ...    27.926414  2.073400e+06

[8 rows x 6 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4902 entries, 0 to 4901
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       4902 non-null   object 
 1   Open       4902 non-null   float64
 2   High       4902 non-null   float64
 3   Low        4902 non-null   float64
 4   Close      4902 non-null   float64
 5   Adj Close  

In [ ]:
stocks['Date']=pd.to_datetime(stocks['Date'])
stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4902 entries, 0 to 4901
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       4902 non-null   datetime64[ns]
 1   Open       4902 non-null   float64       
 2   High       4902 non-null   float64       
 3   Low        4902 non-null   float64       
 4   Close      4902 non-null   float64       
 5   Adj Close  4902 non-null   float64       
 6   Volume     4902 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 268.2 KB


merging stocks and sentiment news dataframe on date

In [ ]:
res = df.merge(stocks, how='inner', on='Date', left_index = True)
res

,headline,Date,neg,neu,pos,compound,Open,High,Low,Close,Adj Close,Volume
0,. Fissures in Hurriyat over Pak visit. America...,2001-01-02,0.124,0.807,0.069,-1,1.437500,1.562500,1.312500,1.437500,1.305893,26600
1,. Think again; Laxman tells Sangh Parivar. Hur...,2001-01-03,0.113,0.771,0.116,1,1.437500,1.437500,1.437500,1.437500,1.305893,18300
2,. Vajpayee gives big push to tech education; R...,2001-01-04,0.092,0.812,0.096,1,1.312500,1.562500,1.312500,1.500000,1.362671,17700
3,. Hawkings' day out. Light combat craft takes ...,2001-01-05,0.124,0.722,0.155,1,1.500000,1.687500,1.500000,1.687500,1.533005,13000
4,. Frontrunner for key US post seen having Pak ...,2001-01-08,0.123,0.809,0.067,-1,1.687500,2.000000,1.687500,2.000000,1.816895,60200
...,...,...,...,...,...,...,...,...,...,...,...,...
4897,. Govt to discuss rules regulating casino busi...,2020-06-22,0.141,0.780,0.079,-1,19.260000,19.260000,18.540001,18.980000,18.906292,203100
4898,. State sugar administration begins issuing sh...,2020-06-23,0.155,0.762,0.083,-1,19.219999,19.219999,18.639999,18.670000,18.597496,91900
4899,. I never thought I had a voice until today: V...,2020-06-24,0.135,0.790,0.075,-1,18.480000,18.540001,18.080000,18.110001,18.039671,160700
4900,. Telangana: Senior Congress leader tests+ve f...,2020-06-25,0.137,0.801,0.062,-1,18.059999,18.059999,17.670000,18.020000,17.950020,127300


In [ ]:
res.isna().sum()

headline     8
Date         0
neg          0
neu          0
pos          0
compound     0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
res=res.dropna()
res.isna().sum()
res

,headline,Date,neg,neu,pos,compound,Open,High,Low,Close,Adj Close,Volume
0,. Fissures in Hurriyat over Pak visit. America...,2001-01-02,0.124,0.807,0.069,-1,1.437500,1.562500,1.312500,1.437500,1.305893,26600
1,. Think again; Laxman tells Sangh Parivar. Hur...,2001-01-03,0.113,0.771,0.116,1,1.437500,1.437500,1.437500,1.437500,1.305893,18300
2,. Vajpayee gives big push to tech education; R...,2001-01-04,0.092,0.812,0.096,1,1.312500,1.562500,1.312500,1.500000,1.362671,17700
3,. Hawkings' day out. Light combat craft takes ...,2001-01-05,0.124,0.722,0.155,1,1.500000,1.687500,1.500000,1.687500,1.533005,13000
4,. Frontrunner for key US post seen having Pak ...,2001-01-08,0.123,0.809,0.067,-1,1.687500,2.000000,1.687500,2.000000,1.816895,60200
...,...,...,...,...,...,...,...,...,...,...,...,...
4897,. Govt to discuss rules regulating casino busi...,2020-06-22,0.141,0.780,0.079,-1,19.260000,19.260000,18.540001,18.980000,18.906292,203100
4898,. State sugar administration begins issuing sh...,2020-06-23,0.155,0.762,0.083,-1,19.219999,19.219999,18.639999,18.670000,18.597496,91900
4899,. I never thought I had a voice until today: V...,2020-06-24,0.135,0.790,0.075,-1,18.480000,18.540001,18.080000,18.110001,18.039671,160700
4900,. Telangana: Senior Congress leader tests+ve f...,2020-06-25,0.137,0.801,0.062,-1,18.059999,18.059999,17.670000,18.020000,17.950020,127300


In [ ]:

y=res['Adj Close']
res=res.drop(columns=['headline','Adj Close','Date','Close'])
res

,neg,neu,pos,compound,Open,High,Low,Volume
0,0.124,0.807,0.069,-1,1.437500,1.562500,1.312500,26600
1,0.113,0.771,0.116,1,1.437500,1.437500,1.437500,18300
2,0.092,0.812,0.096,1,1.312500,1.562500,1.312500,17700
3,0.124,0.722,0.155,1,1.500000,1.687500,1.500000,13000
4,0.123,0.809,0.067,-1,1.687500,2.000000,1.687500,60200
...,...,...,...,...,...,...,...,...
4897,0.141,0.780,0.079,-1,19.260000,19.260000,18.540001,203100
4898,0.155,0.762,0.083,-1,19.219999,19.219999,18.639999,91900
4899,0.135,0.790,0.075,-1,18.480000,18.540001,18.080000,160700
4900,0.137,0.801,0.062,-1,18.059999,18.059999,17.670000,127300


In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
X_scaled = scaler.fit_transform(res)
X_scaled

array([[0.33972603, 0.54694836, 0.30131004, ..., 0.00451712, 0.00766042,
        0.01282917],
       [0.30958904, 0.46244131, 0.50655022, ..., 0.        , 0.01216655,
        0.00882608],
       [0.25205479, 0.55868545, 0.41921397, ..., 0.00451712, 0.00766042,
        0.0085367 ],
       ...,
       [0.36986301, 0.50704225, 0.32751092, ..., 0.61803236, 0.61211247,
        0.07750555],
       [0.37534247, 0.53286385, 0.27074236, ..., 0.60068654, 0.59733237,
        0.06139674],
       [0.43835616, 0.41549296, 0.38864629, ..., 0.59490467, 0.58868061,
        0.11049484]])

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix, classification_report, roc_auc_score,roc_curve,auc
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                            test_size=0.3, random_state=30) 
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)
 
y_pred = reg.predict(X_test)

print('Mean Absolute Error:', 
        mean_absolute_error(y_test, y_pred))
df=pd.DataFrame({'desired':y_test,'predicted':y_pred})
df
#print("Testing Accuracy score: "+str(round(accuracy_score(y_test,y_pred.round()),4)))

Mean Absolute Error: 0.17204288124496023


,desired,predicted
369,3.770057,3.718626
3433,17.502201,17.681513
1459,4.478644,4.407467
1597,7.612790,8.019900
1154,5.814062,5.931503
...,...,...
2076,1.471685,1.274794
436,2.752596,2.550220
3113,16.362921,16.698454
2230,3.742803,3.705037
